The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [4]:
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [5]:
dataset = "a9a"   # news20.binary  rcv1_train.multiclass rcv1_train.binary mnist.scale  a9a  gisette_scale 
prob = defineProb(dataset);
showDetails(dataset)

../data/a9a.jld


In [6]:
norm(prob.A*prob.xsol - prob.b)

1.967333493545081e-11

In [7]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [8]:
options.AUX = [0 2]; # don't use sign flipping and use a density of 10 per row
method_name = "shuffle";# ROS
options.sketchsize = floor((prob.n)^(1/2)); #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "shuffle";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "shuffle";# ROS
options.sketchsize = 5; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "shuffle";# ROS
options.sketchsize = 2; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

shuffle-11-2
-------------------
It   | Error% | Residual |  Time   
-------------------
 96  | 1.35  |  1.88  | 5.2036 
 50  | 0.23  |  0.30  | 5.2143 
 75  | 0.13  |  0.17  | 5.2230 
100  | 0.09  |  0.14  | 5.2287 
125  | 0.07  |  0.13  | 5.2600 
150  | 0.06  |  0.10  | 5.2662 
175  | 0.04  |  0.10  | 5.2759 
200  | 0.03  |  0.07  | 5.2903 
225  | 0.02  |  0.08  | 5.3010 
250  | 0.02  |  0.06  | 5.3064 
275  | 0.01  |  0.05  | 5.3483 
300  | 0.01  |  0.06  | 5.3523 
325  | 0.01  |  0.04  | 5.3712 
350  | 0.01  |  0.03  | 5.3835 
375  | 0.00  |  0.04  | 5.3883 
400  | 0.00  |  0.02  | 5.3923 
425  | 0.00  |  0.03  | 5.4100 
450  | 0.00  |  0.02  | 5.4402 
475  | 0.00  |  0.02  | 5.4507 
500  | 0.00  |  0.02  | 5.4564 
525  | 0.00  |  0.01  | 5.4831 
550  | 0.00  |  0.01  | 5.4872 
575  | 0.00  |  0.01  | 5.4947 
  7.685103 seconds (3.97 M allocations: 194.988 MB, 1.62% gc time)
change rho density to n/s: 2
shuffle-50-5
-------------------
It   | Error% | Residual |  Time   
----------

In [9]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [10]:
# Testing countmin 
# options.sketchsize =10;#floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [11]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 0.03  |  0.03  | 0.3748 
 50  | 0.00  |  0.02  | 0.5884 
 75  | 0.00  |  0.01  | 0.5889 
  0.598565 seconds (445.43 k allocations: 16.520 MB, 1.86% gc time)


In [12]:
# Testing CD 
options.maxiter=5000;
options.skip_error_calculation = 50;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CD";# gradient descent
options.skip_error_calculation = 5;
options.sketchsize = 50;
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 50  | 0.68  |  0.76  | 0.2298 
100  | 0.15  |  0.28  | 0.2322 
150  | 0.11  |  0.30  | 0.2341 
200  | 0.09  |  0.18  | 0.2365 
250  | 0.06  |  0.22  | 0.2381 
300  | 0.05  |  0.13  | 0.2396 
350  | 0.04  |  0.09  | 0.2413 
400  | 0.03  |  0.12  | 0.2428 
450  | 0.02  |  0.08  | 0.2444 
500  | 0.02  |  0.08  | 0.2460 
550  | 0.01  |  0.07  | 0.2476 
600  | 0.00  |  0.04  | 0.2492 
650  | 0.00  |  0.03  | 0.2549 
700  | 0.00  |  0.05  | 0.2566 
750  | 0.00  |  0.01  | 0.2583 
800  | 0.00  |  0.02  | 0.2608 
  0.358588 seconds (108.22 k allocations: 14.951 MB)
CD-50
-------------------
It   | Error% | Residual |  Time   
-------------------
  5  | 2.10  |  2.79  | 0.0060 
 10  | 0.05  |  0.19  | 0.0092 
 15  | 0.02  |  0.10  | 0.0152 
 20  | 0.01  |  0.08  | 0.0221 
 25  | 0.01  |  0.05  | 0.0261 
 30  | 0.00  |  0.02  | 0.0317 
  0.043070 seconds (4.04 k allocations: 2.901 MB)


In [13]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [14]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

a9a
